##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Análisis de modelos mediante canalizaciones de TFX y TensorFlow Model Analysis


Nota: Recomendamos ejecutar este tutorial en un bloc de notas de Colab, ¡no es necesario configurarlo! Simplemente haga clic en "Ejecutar en Google Colab".

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfma"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_tfma.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/penguin_tfma.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/penguin_tfma.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table></div>

En este tutorial basado en un bloc de notas, crearemos y ejecutaremos una canalización de TFX que crea un modelo de clasificación simple y analiza su rendimiento en múltiples ejecuciones. Este bloc de notas se basa en la canalización de TFX que creamos en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). Si aún no ha leído ese tutorial, debe leerlo antes de continuar con este bloc de notas.

A medida que modifica su modelo o lo entrena con un nuevo conjunto de datos, debe verificar si su modelo ha mejorado o empeorado. Tal vez limitarse a comprobar métricas de alto nivel, como la precisión, no sea suficiente. Cada modelo entrenado debe evaluarse antes de insertarse en producción.

Agregaremos un componente `Evaluator` a la canalización creada en el tutorial anterior. El componente Evaluator realiza un análisis profundo de sus modelos y compara el nuevo modelo con una línea base para determinar que sean "lo suficientemente buenos". Se implementa utilizando la biblioteca [TensorFlow Model Analysis](https://www.tensorflow.org/tfx/guide/tfma).

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre varios conceptos en TFX.

## Preparación

El proceso de configuración es el mismo que el del tutorial anterior.

Primero tenemos que instalar el paquete de Python para TFX y descargar el conjunto de datos que usaremos para nuestro modelo.

### Actualización de pip

Para evitar actualizar Pip en un sistema cuando se ejecuta localmente, verifique que se esté ejecutando en Colab. Por supuesto, los sistemas locales se pueden actualizar por separado.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instalación de TFX


In [ ]:
!pip install -U tfx

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "REINICIAR TIEMPO DE EJECUCIÓN" o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

Verifique las versiones de TensorFlow y TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuración de variables

Hay algunas variables que se utilizan para definir una canalización. Puede personalizar estas variables como desee. De forma predeterminada, todas las salidas de la canalización se generarán en el directorio actual.

In [ ]:
import os

PIPELINE_NAME = "penguin-tfma"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparación de datos de ejemplo

Usaremos el mismo [conjunto de datos de Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

Hay cuatro características numéricas en este conjunto de datos que ya fueron normalizadas para tener un rango [0,1]. Compilaremos un modelo de clasificación que prediga las `species` de pingüinos.

Debido a que TFX ExampleGen lee entradas de un directorio, tenemos que crear un directorio y copiar el conjunto de datos en él.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

## Cómo crear una canalización

Agregaremos un componente [`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) a la canalización que creamos en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Un componente Evaluator requiere datos de entrada de un componente `ExampleGen` y un modelo de un componente `Trainer` y un objeto [`tfma.EvalConfig`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig). Opcionalmente, podemos proporcionar un modelo de referencia que se puede utilizar para comparar métricas con el modelo recién entrenado.

Un evaluador crea dos tipos de artefactos de salida, `ModelEvaluation` y `ModelBlessing`. ModelEvaluation contiene la salida de la evaluación detallada que se puede investigar y visualizar más a fondo con la biblioteca TFMA. ModelBlessing contiene una salida booleana que indica si el modelo pasó los criterios establecidos y puede usarse como señal en componentes posteriores, como un Pusher.


### Cómo escribir un código de entrenamiento modelo

Usaremos el mismo código de modelo que en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

### Cómo escribir una definición de canalización

Definiremos una función para crear una canalización de TFX. Además del componente Evaluator que mencionamos anteriormente, agregaremos un nodo más llamado [`Resolver`](https://www.tensorflow.org/tfx/api_docs/python/tfx/v1/dsl/Resolver). Para comprobar que un nuevo modelo es mejor que el modelo anterior, debemos compararlo con un modelo publicado anteriormente, conocido como línea base. [ML Metadata (MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) rastrea todos los artefactos anteriores de la canalización y `Resolver` puede encontrar cuál fue el último modelo *con visto bueno* (un modelo aprobado con éxito por Evaluator) de MLMD con una clase de estrategia llamada `LatestBlessedModelStrategy`.


In [ ]:
import tensorflow_model_analysis as tfma

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # NEW: Get the latest blessed model for Evaluator.
  model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

  # NEW: Uses TFMA to compute evaluation statistics over features of a model and
  #   perform quality validation of a candidate model (compared to a baseline).

  eval_config = tfma.EvalConfig(
      model_specs=[tfma.ModelSpec(label_key='species')],
      slicing_specs=[
          # An empty slice spec means the overall slice, i.e. the whole dataset.
          tfma.SlicingSpec(),
          # Calculate metrics for each penguin species.
          tfma.SlicingSpec(feature_keys=['species']),
          ],
      metrics_specs=[
          tfma.MetricsSpec(per_slice_thresholds={
              'sparse_categorical_accuracy':
                  tfma.PerSliceMetricThresholds(thresholds=[
                      tfma.PerSliceMetricThreshold(
                          slicing_specs=[tfma.SlicingSpec()],
                          threshold=tfma.MetricThreshold(
                              value_threshold=tfma.GenericValueThreshold(
                                   lower_bound={'value': 0.6}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}))
                       )]),
          })],
      )
  evaluator = tfx.components.Evaluator(
      examples=example_gen.outputs['examples'],
      model=trainer.outputs['model'],
      baseline_model=model_resolver.outputs['model'],
      eval_config=eval_config)

  # Checks whether the model passed the validation steps and pushes the model
  # to a file destination if check passed.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      model_blessing=evaluator.outputs['blessing'], # Pass an evaluation result.
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      trainer,

      # Following two components were added to the pipeline.
      model_resolver,
      evaluator,

      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

Necesitamos proporcionar la siguiente información a Evaluator a través de `eval_config`:

- Métricas adicionales para configurar (si desea más métricas que las definidas en el modelo).
- Segmentos para configurar
- Umbrales de validaciones del modelo para verificar si se incluirá la validación

Como `SparseCategoricalAccuracy` ya estaba incluido en la llamada `model.compile()`, se incluirá en el análisis automáticamente. Por lo tanto, no agregamos ninguna métrica adicional aquí. `SparseCategoricalAccuracy` se usará para decidir si el modelo también es lo suficientemente bueno.

Calculamos las métricas para todo el conjunto de datos y para cada especie de pingüino. `SlicingSpec` especifica cómo se agregan las métricas declaradas.

Hay dos umbrales que debe superar un nuevo modelo, uno es un umbral absoluto de 0,6 y el otro es un umbral relativo que debe ser más alto que el modelo de línea base. Cuando ejecute la canalización por primera vez, se ignorará `change_threshold` y solo se comprobará value_threshold. Si ejecuta la canalización más de una vez, `Resolver` encontrará un modelo de la ejecución anterior y lo usará como modelo de línea base para la comparación.

Consulte la [guía del componente Evaluator](https://www.tensorflow.org/tfx/guide/evaluator#using_the_evaluator_component) para obtener más información.

## Cómo ejecutar la canalización


Usaremos `LocalDagRunner` como en el tutorial anterior.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Cuando se complete la canalización, debería poder ver algo como lo siguiente:

```
INFO:absl:Blessing result True written to pipelines/penguin-tfma/Evaluator/blessing/4.
```

O también puede verificar manualmente el directorio de salida donde se almacenan los artefactos generados. Si visita `pipelines/penguin-tfma/Evaluator/blessing/` con un navegador de archivos, puede ver un archivo con un nombre `BLESSED` o `NOT_BLESSED` según el resultado de la evaluación.

Si el resultado de la aprobación es `False`, Pusher se negará a insertar el modelo a `serving_model_dir` porque el modelo no es lo suficientemente bueno para usarlo en producción.

Puede ejecutar la canalización nuevamente, posiblemente con diferentes configuraciones de evaluación. Incluso si ejecuta la canalización con exactamente la misma configuración y conjunto de datos, el modelo entrenado puede ser ligeramente diferente debido a la aleatoriedad inherente del entrenamiento del modelo que puede conducir a un modelo `NOT_BLESSED`.

### Análisis de salidas de la canalización

Puede usar TFMA para investigar y visualizar el resultado de la evaluación en el artefacto ModelEvaluation.

> **NOTA: Si no está en Colab, instale las extensiones de Jupyter.** Necesita una extensión de TensorFlow Model Analysis para ver la visualización de TFMA. Esta extensión ya está instalada en Google Colab, pero es posible que deba instalarla si ejecuta este bloc de notas en otros entornos. Consulte la dirección de instalación de la extensión de Jupyter en la [Guía de instalación](https://github.com/tensorflow/model-analysis#installation).


#### Obtención del resultado del análisis de los artefactos de salida

Puede utilizar las API de MLMD para localizar estas salidas mediante programación. Primero, definiremos algunas funciones de utilidad para buscar artefactos de salida que se acaban de producir.

In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)


Podemos encontrar la última ejecución del componente `Evaluator` y obtener artefactos de salida de él.

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  evaluator_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                          'Evaluator')
  eval_artifact = evaluator_output[standard_component_specs.EVALUATION_KEY][0]

`Evaluator` siempre devuelve un artefacto de evaluación y podemos visualizarlo usando la biblioteca TensorFlow Model Analysis. Por ejemplo, el siguiente código mostrará las métricas de precisión para cada especie de pingüino.

In [ ]:
import tensorflow_model_analysis as tfma

eval_result = tfma.load_eval_result(eval_artifact.uri)
tfma.view.render_slicing_metrics(eval_result, slicing_column='species')

Si elige 'sparse_categorical_accuracy' en la lista desplegable `Show`, podrá ver los valores de precisión por especie. Tal vez le convenga agregar más segmentos y verificar si su modelo es bueno para todas las distribuciones y si existe algún posible sesgo.

## Siguientes pasos

Obtenga más información sobre el análisis de modelos en el [tutorial de la biblioteca TensorFlow Model Analysis](https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

Puede encontrar más recursos en https://www.tensorflow.org/tfx/tutorials.

Consulte [Explicación de las canalizaciones de TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para obtener más información sobre varios conceptos en TFX.
